In [1]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFKC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(files=paths, trainer=trainer)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)



In [3]:
file1 = open('Atikamekw_corpus.txt', 'r')
Lines = file1.readlines()
print (Lines[1:3])
# count = 0
all_text= []
# # Strips the newline character
for line in Lines:
    all_text.append(line.split('$$$')[1][:-2])
    #count += 1
    #print("Line{}: {}".format(count, line.strip()))
    
print (all_text[1:3])

['%C3%89cole$$$otapi kiskinohamatowikam part_prox ici acteam atikam oten manawan icinikatcikateam part_rept nicw mitatw mitanw part_conn niar mitanw taciw kiskinohamew otatako pisimam part_sub part_sub aikatew part_sub tcimatakiniw anihe kiskinohamatowikam teciko Pron mi matce pitakew awac part_sub kiskinohamaso wipaw Pron iti kiskeritako part_sub osamew apicac atikamew part_time Pron mi nataweritam part_sub rikitakaniwon okiskinohamatowikamew part_sub iti ocitakiniw part_time mitcew miriwewaniwo kiskinohamakew emitikocimow ekarecamow akitasow tesinikew mirokinehitisow sciences atis e \n', '%C3%8Ele$$$cap otenw aski ici acteam kanatw irik part_sub tacinani \n']
['otapi kiskinohamatowikam part_prox ici acteam atikam oten manawan icinikatcikateam part_rept nicw mitatw mitanw part_conn niar mitanw taciw kiskinohamew otatako pisimam part_sub part_sub aikatew part_sub tcimatakiniw anihe kiskinohamatowikam teciko Pron mi matce pitakew awac part_sub kiskinohamaso wipaw Pron iti kiskeritako pa

In [4]:
paths = ['Atikamekw_corpus.txt']
print (paths)

tokenizer = BPE_token()
# train the tokenizer model
tokenizer.bpe_train(paths)

# saving the tokenized data in our specified folder 
save_path = './Atikamekw_bot/'
tokenizer.save_tokenizer(save_path)


['Atikamekw_corpus.txt']


In [5]:
# #from tokenise import BPE_token
# from pathlib import Path
# import os 
# # the folder 'text' contains all the files
# paths = [str(x) for x in Path("./Wikipedia Articales Processed/").glob("**/*.txt")]

# print (paths[:3])

# tokenizer = BPE_token()
# # train the tokenizer model
# tokenizer.bpe_train(paths)

# # saving the tokenized data in our specified folder 
# save_path = './Atikamekw_bot/tokenized_data'
# tokenizer.save_tokenizer(save_path)

In [6]:
import random

n = int(len(all_text)/10)

random.shuffle(all_text)

test_data = all_text[:n]
train_data = all_text[n:]

print (len(train_data))

print (len(test_data))

# printing n elements from list


1426
158


In [7]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
model = TFGPT2LMHeadModel(config)

In [8]:
# single_string = ''
# for filename in train_data:
#     with open(filename, "r", encoding='utf-8') as f:
#         x = f.read()
#     single_string += x + tokenizer.eos_token
# string_tokenized = tokenizer.encode(single_string)
single_string = ''
for file in train_data:
    single_string += file + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [9]:
examples = []
block_size = 100
BATCH_SIZE = 8
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
    examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [11]:
num_epoch = 100
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/100
197/197 [==============================] - 50s 139ms/step - loss: 5.2432 - accuracy: 0.3746
Epoch 2/100
197/197 [==============================] - 27s 138ms/step - loss: 4.2844 - accuracy: 0.4126
Epoch 3/100
197/197 [==============================] - 27s 140ms/step - loss: 4.0700 - accuracy: 0.4262
Epoch 4/100
197/197 [==============================] - 28s 141ms/step - loss: 3.9286 - accuracy: 0.4343
Epoch 5/100
197/197 [==============================] - 27s 139ms/step - loss: 3.8015 - accuracy: 0.4412
Epoch 6/100
197/197 [==============================] - 27s 138ms/step - loss: 3.6804 - accuracy: 0.4484
Epoch 7/100
197/197 [==============================] - 27s 139ms/step - loss: 3.5628 - accuracy: 0.4565
Epoch 8/100
197/197 [==============================] - 29s 149ms/step - loss: 3.4511 - accuracy: 0.4657
Epoch 9/100
197/197 [==============================] - 27s 139ms/step - loss: 3.3380 - accuracy: 0.4752
Epoch 10/100
197/197 [==============================] - 28s 140m

197/197 [==============================] - 28s 140ms/step - loss: 0.0871 - accuracy: 0.9833
Epoch 80/100
197/197 [==============================] - 27s 139ms/step - loss: 0.0853 - accuracy: 0.9832
Epoch 81/100
197/197 [==============================] - 29s 145ms/step - loss: 0.0850 - accuracy: 0.9835
Epoch 82/100
197/197 [==============================] - 28s 142ms/step - loss: 0.0822 - accuracy: 0.9837
Epoch 83/100
197/197 [==============================] - 27s 139ms/step - loss: 0.0803 - accuracy: 0.9836
Epoch 84/100
197/197 [==============================] - 27s 139ms/step - loss: 0.0790 - accuracy: 0.9842
Epoch 85/100
197/197 [==============================] - 27s 139ms/step - loss: 0.0786 - accuracy: 0.9840
Epoch 86/100
197/197 [==============================] - 27s 138ms/step - loss: 0.0778 - accuracy: 0.9838
Epoch 87/100
197/197 [==============================] - 27s 137ms/step - loss: 0.0756 - accuracy: 0.9842
Epoch 88/100
197/197 [==============================] - 27s 139ms/st

In [ ]:
# text = "Otapi kiskinohamatowikamikw"
# # encoding the input text
# input_ids = tokenizer.encode(text, return_tensors='tf')
# # getting out output
# beam_output = model.generate(
#   input_ids,
#   max_length = 50,
#   num_beams = 5,
#   temperature = 0.7,
#   no_repeat_ngram_size=2,
#   num_return_sequences=5
# )

In [ ]:
# print (tokenizer.decode(beam_output[0],skip_special_tokens = True))


In [12]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
output_dir = './Atikamekw_bot/'
# creating directory if it is not present
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

model_to_save = model.module if hasattr(model, 'module') else model

output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
model.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained(output_dir)

('./Atikamekw_bot/tokenizer_config.json',
 './Atikamekw_bot/special_tokens_map.json',
 './Atikamekw_bot/vocab.json',
 './Atikamekw_bot/merges.txt',
 './Atikamekw_bot/added_tokens.json')

In [13]:
with open ('./Atikamekw_bot/test_original_corpus.txt', 'w',encoding='utf-8') as f:
    for line in test_data:
        f.write(f"{line}\n")

In [14]:
import glob
import tqdm

In [22]:
def generate_Text_bot(test_data,bot_path):
    
    output_file = './Atikamekw_bot/test_bot_corpus.txt'
    tokenizer = GPT2Tokenizer.from_pretrained(bot_path)
    model = TFGPT2LMHeadModel.from_pretrained(bot_path)
    for line in tqdm.tqdm(test_data):
        start_token = ' '.join(line.split()[:2])
        print (start_token)
        input_ids = tokenizer.encode(start_token, return_tensors='tf')
        # getting out output
        beam_output = model.generate(
          input_ids,
          max_length = len(line.split()),
          num_beams = 5,
          temperature = 0.7,
          no_repeat_ngram_size=2,
          num_return_sequences=5
        )
        output_text = tokenizer.decode(beam_output[0],skip_special_tokens = True)
        print (output_text)
        with open(output_file, 'a',encoding='utf-8') as f:
            f.write(f"{output_text}\n")
            
        #break

In [23]:
generate_Text_bot(test_data,'./Atikamekw_bot/')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./Atikamekw_bot/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
  0%|          | 0/158 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


enkw w_demo


  1%|          | 1/158 [00:03<08:55,  3.41s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


enkw w_demo ocki irapitcitc part_sub apit kaskinw wec ici otamironaniw
enkw part_sub


  1%|▏         | 2/158 [00:07<10:04,  3.88s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


...

kaasisiwakam sakih kepew ask ici acteam kanat
atikam metowanikanw


 99%|█████████▉| 157/158 [39:23<00:16, 16.62s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


atikam metowanikanw awaci metow kaki mitw ocitaw atikam nehirowisw mekwaw part_sub metowehaso onitcanicw otci matceatisiw notc Pron otinam otci onitcanicw part pisima
awtosw sakih


100%|██████████| 158/158 [39:25<00:00, 14.97s/it]

awtosw sakih kepew ask ici acteam kanat


In [18]:
# def generate_Text_bot(input_path,bot_path,test_data):
    
#     output_folder = './Atikamekw_bot/generated_text_unprocessed'
#     if not os.path.exists(output_folder):
#         os.mkdir(output_folder)
#     files = sorted(glob.glob(input_path + '/*'))
#     tokenizer = GPT2Tokenizer.from_pretrained(bot_path)
#     model = TFGPT2LMHeadModel.from_pretrained(bot_path)
#     for file in tqdm.tqdm(files):
        
#         if file not in test_data:
#             print ('1')
#             continue
#         print (file)
#         file_name  = file.split('\\')[-1]
#         output_file = output_folder + '/bot_generated_' + file_name
#         file = open(file ,mode='r',encoding = 'utf-8')
#         all_of_it = file.read()
        
#         start_token = ' '.join(all_of_it.split()[:2])
#         print (start_token)
        
#         input_ids = tokenizer.encode(start_token, return_tensors='tf')
#         # getting out output
#         beam_output = model.generate(
#           input_ids,
#           max_length = len(all_of_it),
#           num_beams = 5,
#           temperature = 0.7,
#           no_repeat_ngram_size=2,
#           num_return_sequences=5
#         )
#         output_text = tokenizer.decode(beam_output[0],skip_special_tokens = True)
#         with open(output_file, 'w',encoding='utf-8') as f:
#             f.write(f"{output_text[:-1]}\n")
            
#         break

In [ ]:
import tensorflow as tf

In [ ]:
print("Num GPUs Available: ", tf.config.list_physical_devices())

In [ ]:
tf.debugging.set_log_device_placement(False)

In [ ]:
print(tf. __version__) 